In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [5]:
train.head()

In [6]:
test.shape, train.shape

In [7]:
train.info()

## Очистка данных

In [8]:
train.isna().sum()

In [9]:
 test.isna().sum()

In [10]:
train.fillna(method="bfill", inplace=True)
test.fillna(method="bfill", inplace=True)

In [11]:
train.isna().sum()

## EDA

In [12]:
from pandas_profiling import ProfileReport

profrep = ProfileReport(train, title='Pandas Profiling Report', explorative=True)
profrep

In [13]:
! pip install sweetviz

In [14]:
import sweetviz as sv

sv_report = sv.analyze([train, 'Passangers info'], target_feat='Transported')

In [15]:
sv_report.show_notebook()

In [16]:
report = sv.compare(train,test,'Age')
report.show_notebook()

In [17]:
report.show_html('age_comparison')

## Рассмотрим категориальные признаки

In [18]:
train.astype({'Transported': "O"}).dtypes

In [19]:
categorical_features = []
numerical_features = []

for column in train.columns:
    if train[column].dtypes == 'O':
        categorical_features.append(column)
    else:
        numerical_features.append(column)

In [20]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for feature in categorical_features:
    train[feature] = le.fit_transform(train[feature])

In [21]:
train.head()

In [22]:
for category in categorical_features:
    if category !='Transported':
        test[category] = le.fit_transform(test[category])
test.head()

In [23]:
X_train = train.drop('Transported',1)
y_train = train.Transported

In [24]:
X_train

In [25]:
y_train

## XGBoost Model

In [29]:
! pip install autokeras

In [30]:
from xgboost import XGBClassifier
import autokeras as ak

XGB_model = XGBClassifier(n_estimators=1000, learning_rate=0.05, n_jobs=-1,cv=ak)
XGB_model.fit(X_train, y_train)

In [36]:
XGB_model.score(X_train, y_train)

In [32]:
prediction = XGB_model.predict(test)

In [33]:
prediction

In [33]:
submission =  pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
submission['Transported'] = prediction
submission['Transported'] = submission['Transported'].replace({1: True, 0: False})
submission.set_index('PassengerId',inplace=True)

submission.head()

In [34]:
submission.to_csv('submission.csv')

## CatBoost Model

In [45]:
from catboost import CatBoostClassifier

CatBoost_model = CatBoostClassifier()
y_train.replace({1: True, 0: False})

CatBoost_model.fit(X_train, y_train)

In [46]:
CatBoost_model.score(X_train, y_train)

In [44]:
CatBoost_model.predict(test)